<a href="https://colab.research.google.com/github/khush2520/Word-Sense-Disambiguation/blob/main/WSD1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [524]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [525]:
from google.colab import drive
drive.mount('/content/drive')
xml_file = 'drive/MyDrive/semcor.data.xml'
gold_key = 'drive/MyDrive/semcor.gold.key.txt'
test_xml_file = 'drive/MyDrive/ALL.data.xml'
test_gold_key = 'drive/MyDrive/ALL.gold.key.txt'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [526]:
import xml.etree.ElementTree as ET

tree = ET.parse(xml_file)
root = tree.getroot()
# print(type(root))

In [527]:
lines = []
with open(gold_key, 'r') as file:
    lines = file.readlines()

Y_values = {}
processed_data = []
for line in lines:
    words = line.strip().split()
    processed_data.append(words[1])
    Y_values[words[0]] = words[1]
#     print(item)
Y = processed_data

print(len(processed_data))
print(len(set(processed_data)))
# print(Y_values)

226036
33316


In [528]:
file_path = test_gold_key
lines = []
with open(test_gold_key, 'r') as file:
    lines = file.readlines()
# print(lines[0:9])

test_Y_values = {}
test_processed_data = []
for line in lines:
    words = line.strip().split()
    test_processed_data.append(words[1])
    test_Y_values[words[0]] = words[1]
    # pos = words[0].find('%')

    # processed_data.append(words[0][:pos])

test_Y = test_processed_data
# print(test_Y_values)
print(len(test_processed_data))
print(len(set(test_processed_data)))
# print(Y_values)

7253
3520


In [529]:
dataset = {}
for sentence in root.findall('.//sentence'):
    # print(f"Sentence ID: {sentence.attrib['id']}")
    sent = ""
    for element in sentence:
            sent = sent + " " + element.text

    for element in sentence:
        if element.tag in ['instance']:
              lemma = element.attrib.get('lemma')
              id = element.attrib.get('id')
              pos = element.attrib.get('pos')
              if lemma not in dataset:
                dataset[lemma] = []
              dataset[lemma].append([id,lemma, sent, pos, element.text, ])

    # print(sent)

print(len(dataset))


20399


In [530]:
for lemma in dataset:
  for j in dataset[lemma]:
    y = Y_values[j[0]]
    pos = y.index('%')
    j.append(y[pos:])


In [531]:

test_tree = ET.parse(test_xml_file)
test_root = test_tree.getroot()
# print(type(root))
test_dataset = {}
for sentence in test_root.findall('.//sentence'):
    # print(f"Sentence ID: {sentence.attrib['id']}")
    sent = ""
    for element in sentence:
            sent = sent + " " + element.text
    for element in sentence:
            if element.tag in ['instance']:
              lemma = element.attrib.get('lemma')
              id = element.attrib.get('id')
              pos = element.attrib.get('pos')
              if lemma not in test_dataset:
                test_dataset[lemma] = []
              test_dataset[lemma].append([id,lemma, sent,pos,element.text])
            # print(f"{element.text} Lemma: {lemma}, POS: {pos}")
    # print(sent)

# print(len(test_dataset))
# print(features)


In [532]:
for lemma in test_dataset:
  for j in test_dataset[lemma]:
    y = test_Y_values[j[0]]
    pos = y.index('%')
    j.append(y[pos:])


In [533]:
from nltk.corpus import wordnet as wn
POS  = {'n':'NOUN', 'a': 'ADJ','s': 'ADJ', 'v': 'VERB', 'r':'ADV'}
def add_dataset_from_wordnet(word):
    word2 = word.replace('_',' ')
    word2 = word2.replace('-',' ')
    # print("word", word)
    synsets = wn.synsets(word)
    if synsets:
      for word_synset in synsets:
        lemma = word_synset.lemmas()[0].name()
        example_sentence = word_synset.examples()[0] if word_synset.examples() else word_synset.definition()

        p = word_synset.pos()
        pos = POS[p]

        # Find the word form in the example sentence
        word_form = None
        if example_sentence is not None:
          word_forms = example_sentence.split()
          for wf in word_forms:
            wf_synsets = wn.synsets(wf)
            for wf_synset in wf_synsets:
                wf_lemma = wf_synset.lemmas()[0].name()
                # print(wf_l)
                if lemma == wf_lemma:
                  word_form = wf
                  break;
            if word_form:
              break
        for sense in word_synset.lemmas():
          sense_key = sense.key()
          posi = sense_key.index('%')
          if sense_key.startswith(word+'%'):
            posi = sense_key.index('%')
            break

        if word_form is None:
          word_form = word
        if word not in dataset:
          dataset[word] = []
        dataset[word].append([None, lemma, example_sentence, pos, word_form, sense_key[posi:]])
        # print('hi', [lemma, example_sentence, pos, word_form, sense_key])
    else:
        print('hi',word, [None])



In [534]:
words = list(test_dataset.keys())

for lemma in words:
    add_dataset_from_wordnet(lemma)

In [535]:
words = list(test_dataset.keys())
total_accuracy = 0
lc = 0
for lemma in words:
    if lemma not in dataset:
      print(lemma)
      continue

    X_train = [row[1:-1] for row in dataset[lemma]]
    y_train = [row[-1] for row in dataset[lemma]]


    X_test = [row[1:-1] for row in test_dataset[lemma]]
    y_test = [row[-1] for row in test_dataset[lemma]]
    vectorizer = DictVectorizer(sparse=False)
    # X_train_vectorized = vectorizer.fit_transform([dict(zip(['pos', 'element.text'], x)) for x in X_train])
    X_train_vectorized = vectorizer.fit_transform([dict(zip(['lemma', 'sent', 'pos', 'element.text'], x)) for x in X_train])

    naive_bayes_classifier = MultinomialNB()

    naive_bayes_classifier.fit(X_train_vectorized, y_train)

    # X_test_vectorized = vectorizer.transform([dict(zip(['pos', 'element.text'], x)) for x in X_test])
    X_test_vectorized = vectorizer.transform([dict(zip(['lemma', 'sent', 'pos', 'element.text'], x)) for x in X_test])
    predictions = naive_bayes_classifier.predict(X_test_vectorized)
    accuracy = naive_bayes_classifier.score(X_test_vectorized, y_test)

    if len(set(y_train)) < 2:
        # if list(set(test_y))[0] != list(set(train_y))[0]:
        #     print(lemma, test_y, list(set(train_y))[0])

        lc += len(y_test)
        total_accuracy += 1 * len(X_test)
        continue
    # print("Accuracy:", accuracy)
    lc+= len(X_test)
    total_accuracy += accuracy * len(X_test)
print(total_accuracy/lc)


0.6321522128774301


In [536]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack

words = list(test_dataset.keys())
total_accuracy = 0
lc = 0

vectorizer = TfidfVectorizer()  # Initialize vectorizer outside the loop

for lemma in words:
    if lemma not in dataset:
        print(lemma)
        continue

    train_data = dataset[lemma]
    test_data = test_dataset[lemma]

    train_features = [row[1:5] for row in train_data]  # Features: lemma, sent, pos, element.text
    train_y = [row[5] for row in train_data]  # Target 'y'

    test_features = [row[1:5] for row in test_data]
    test_y = [row[5] for row in test_data]

    train_texts = [' '.join(row) for row in train_features]
    test_texts = [' '.join(row) for row in test_features]

    train_vectorized = vectorizer.fit_transform(train_texts)
    test_vectorized = vectorizer.transform(test_texts)

    model = SVC(kernel='linear', gamma='auto')

    if len(set(train_y)) < 2:
        # if list(set(test_y))[0] != list(set(train_y))[0]:
        #     print(lemma, test_y, list(set(train_y))[0])

        lc += len(test_texts)
        total_accuracy += 1 * len(test_texts)
        continue

    model.fit(train_vectorized, train_y)
    predicted_y = model.predict(test_vectorized)

    accuracy = accuracy_score(test_y, predicted_y)
    lc += len(test_texts)
    total_accuracy += accuracy * len(test_texts)


print(total_accuracy / lc)


0.6211222942230801


In [538]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction import DictVectorizer

words = list(test_dataset.keys())
total_accuracy = 0
lc = 0

for lemma in words:
    if lemma not in dataset:
        print(lemma)
        continue

    X_train = [row[1:-1] for row in dataset[lemma]]
    y_train = [row[-1] for row in dataset[lemma]]

    X_test = [row[1:-1] for row in test_dataset[lemma]]
    y_test = [row[-1] for row in test_dataset[lemma]]

    if len(set(y_train)) < 2:
        lc += len(y_test)
        total_accuracy += 1 * len(X_test)
        continue

    vectorizer = DictVectorizer(sparse=False)
    X_train_vectorized = vectorizer.fit_transform([dict(zip(['lemma', 'sent', 'pos', 'element.text'], x)) for x in X_train])

    knn_classifier = KNeighborsClassifier(n_neighbors=(max(2,len(y_train)//3)))  # You can adjust n_neighbors as needed

    knn_classifier.fit(X_train_vectorized, y_train)

    X_test_vectorized = vectorizer.transform([dict(zip(['lemma', 'sent', 'pos', 'element.text'], x)) for x in X_test])

    predictions = knn_classifier.predict(X_test_vectorized)
    accuracy = knn_classifier.score(X_test_vectorized, y_test)


    lc += len(X_test)
    total_accuracy += accuracy * len(X_test)

print(total_accuracy / lc)


0.6211222942230801


In [539]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction import DictVectorizer

words = list(test_dataset.keys())
total_accuracy = 0
lc = 0

for lemma in words:
    if lemma not in dataset:
        print(lemma)
        continue

    X_train = [row[1:-1] for row in dataset[lemma]]
    y_train = [row[-1] for row in dataset[lemma]]

    X_test = [row[1:-1] for row in test_dataset[lemma]]
    y_test = [row[-1] for row in test_dataset[lemma]]

    if len(set(y_train)) < 2:
        lc += len(y_test)
        total_accuracy += 1 * len(X_test)
        continue

    vectorizer = DictVectorizer(sparse=False)
    X_train_vectorized = vectorizer.fit_transform([dict(zip(['lemma', 'sent', 'pos', 'element.text'], x)) for x in X_train])

    adaboost_classifier = AdaBoostClassifier(n_estimators=50, random_state=42)  # You can adjust n_estimators as needed

    adaboost_classifier.fit(X_train_vectorized, y_train)

    X_test_vectorized = vectorizer.transform([dict(zip(['lemma', 'sent', 'pos', 'element.text'], x)) for x in X_test])

    predictions = adaboost_classifier.predict(X_test_vectorized)
    accuracy = adaboost_classifier.score(X_test_vectorized, y_test)

    lc += len(X_test)
    total_accuracy += accuracy * len(X_test)

print(total_accuracy / lc)


0.6347718185578382
